In [75]:
print("Hello, starting...")
# Installation of Required Libraries
!pip install transformers datasets torch tensorflow-macos tensorflow-metal evaluate peft seaborn

import torch
import time
import tensorflow as tf
import os

# Check if MPS (Metal Performance Shaders) is available
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

print("Library installed.")

if torch.backends.mps.is_available():
    print("MPS backend is available!")
else:
    print("MPS backend is not available.")

devices = tf.config.experimental.list_physical_devices('GPU')
print("Devices: ", devices)
    

Hello, starting...


python3.11(85974) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Library installed.
MPS backend is available!
Devices:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [76]:
import os

#os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"]="0.0"

In [77]:
# Check if MPS is available and set the device
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print("Device: ", device)

Device:  mps


In [78]:
import numpy as np
import torch
import evaluate
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns

# Check MPS availability
if not torch.backends.mps.is_available():
    raise ValueError("MPS device not found. Please ensure you are running this on a MacBook Pro with MPS support.")

# Load dataset
dataset = load_dataset("imdb")

# Split dataset
dataset_split = dataset['train'].train_test_split(test_size=0.1).rename_column('label', 'labels')
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']
test_dataset = dataset['test'].rename_column('label', 'labels')

# Load tokenizer and model
model_name = "roberta-large"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

# LoRA Configuration
config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)

# Apply LoRA to the model
model = get_peft_model(model, config)

# Tokenize function
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Debugging step: Print a sample from the tokenized datasets
print("tokenized_train: ", tokenized_train[0])
print("tokenized_eval: ", tokenized_eval[0])
print("tokenized_test: ", tokenized_test[0])

# Ensure the tokenized datasets contain the correct columns
tokenized_train = tokenized_train.remove_columns(["text"]).with_format("torch")
tokenized_eval = tokenized_eval.remove_columns(["text"]).with_format("torch")
tokenized_test = tokenized_test.remove_columns(["text"]).with_format("torch")

# Check the columns after formatting
print("tokenized_train.column_names: ", tokenized_train.column_names)
print("tokenized_eval.column_names: ", tokenized_eval.column_names)
print("tokenized_test.column_names: ", tokenized_test.column_names)

print("assert start...")
# Ensure the required fields are present
assert "input_ids" in tokenized_train.column_names
assert "attention_mask" in tokenized_train.column_names
assert "labels" in tokenized_train.column_names
print("assert completed!")

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-6,  # Reduced learning rate
    per_device_train_batch_size=8,  # Reduced batch size
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    warmup_steps=500,
    gradient_accumulation_steps=2,
    disable_tqdm=False,
    report_to=[],
    logging_dir="./logs",
    logging_steps=10,
    max_grad_norm=1.0  # Gradient clipping
)

# Metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_metric.compute(predictions=preds, references=p.label_ids)["accuracy"]
    f1 = f1_metric.compute(predictions=preds, references=p.label_ids, average="weighted")["f1"]
    return {"accuracy": acc, "f1": f1}

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train and evaluate
trainer.train()

# Push model to Hugging Face Hub
trainer.push_to_hub("finetuned-roberta-large-imdb-LORA")

# Evaluate on test dataset
predictions = trainer.predict(tokenized_test)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

# Confusion matrix
cm = confusion_matrix(labels, preds)
accuracy = accuracy_score(labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title(f"Confusion Matrix\nAccuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")
plt.show()

# Display some false positives and false negatives
false_positives = np.where((preds == 1) & (labels == 0))[0]
false_negatives = np.where((preds == 0) & (labels == 1))[0]

print("Sample False Positives:")
for idx in false_positives[:3]:
    print(f"Review: {test_dataset[idx]['text']}\nPredicted: {preds[idx]}, Actual: {labels[idx]}\n")

print("Sample False Negatives:")
for idx in false_negatives[:3]:
    print(f"Review: {test_dataset[idx]['text']}\nPredicted: {preds[idx]}, Actual: {labels[idx]}\n")

# Inference function
def infer(text):
    # Load model from Hugging Face Hub
    model = RobertaForSequenceClassification.from_pretrained("jigarcpatel/finetuned-roberta-large-imdb-LORA")
    tokenizer = RobertaTokenizer.from_pretrained("jigarcpatel/finetuned-roberta-large-imdb-LORA")

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to('mps') if torch.backends.mps.is_available() else v for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return torch.argmax(probs, dim=-1).item()

# Sample inference
sample_texts = [
    "This product is great! I love it and would highly recommend it to anyone.",
    "Terrible service. I am very disappointed and will not be coming back."
]

for text in sample_texts:
    prediction = infer(text)
    print(f"Text: {text}\nPrediction: {'Positive' if prediction == 1 else 'Negative'}\n")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

tokenized_train:  {'text': "With these people faking so many shots, using old footage, and gassing animals to get them out, not to mention that some of the scenes were filmed on a created set with actors, what's to believe? Old film of countries is nice, but the animal abuse and degradation of natives is painful to watch in these films. I know, racism is OK in these old films, but there is more to that to make this couple lose credibility. Portrayed as fliers, they never flew their planes, Martin Johnson was an ex-vaudevillian, used friends like Jack London for financial gain while stiffing them of royalties, denying his wife's apparent depression, using her as a cute prop, all this makes these films unbearable. They were by no means the first to travel to these lands, or the first to write about them. He was OK as a filmmaker and photographer, but that's about it.", 'labels': 0, 'input_ids': [0, 3908, 209, 82, 856, 7520, 98, 171, 2347, 6, 634, 793, 4338, 6, 8, 821, 13613, 3122, 7, 120

IndexError: Invalid key: 19462 is out of bounds for size 0